In [ ]:
import obi_one as obi
import os
import pandas as pd

In [ ]:
input_root = "/Users/pokorny/Data/Circuits"
output_root = "../../../obi-output/extracted_single_neuron_circuits"

In [ ]:
# Number of single-neuron circuits to extract
idx_list = [0, 1]
node_selection_file = "./top_nodes_dim6.csv"
node_id_lists = [obi.NamedTuple(name=f"SingleNeuronCircuit__" + os.path.splitext(os.path.split(node_selection_file)[1])[0] + f"__IDX{_idx}", elements=tuple([pd.read_csv(node_selection_file)["node_ids"].to_list()[_idx]])) for _idx in idx_list]

In [ ]:
circuit_extractions_scan_config = obi.CircuitExtractionScanConfig(
                    initialize=obi.CircuitExtractionScanConfig.Initialize(
                        # circuit=obi.Circuit(name="ToyCircuit-S1-6k", path=os.path.join(input_root, "ToyCircuit-S1-6k", "circuit_config.json")),
                        circuit=obi.Circuit(name="nbS1-HEX0", path=os.path.join(input_root, "nbS1-HEX0", "circuit_config.json")),
                        run_validation=False,
                    ),
                    # neuron_set=obi.PredefinedNeuronSet(node_set=["Excitatory", "Inhibitory"], sample_percentage=[50, 100]),
                    # neuron_set=obi.PredefinedNeuronSet(node_set=["Layer2", "Layer3", "Layer4", "Layer5", "Layer6"], sample_percentage=10),
                    neuron_set=obi.IDNeuronSet(neuron_ids=node_id_lists)
)

grid_scan = obi.GridScanGenerationTask(form=circuit_extractions_scan_config, output_root=output_root, coordinate_directory_option="VALUE")

In [ ]:
# Display the intermediary data
grid_scan.multiple_value_parameters(display=True)
grid_scan.coordinate_parameters(display=True)
# grid_scan.coordinate_instances(display=True)

In [ ]:
grid_scan.execute()
obi.run_tasks_for_generated_scan(grid_scan)

In [ ]:
# Check circuits
from bluepysnap import Circuit

for _inst in grid_scan.coordinate_instances():
    cfg = grid_scan.output_root / str(_inst.neuron_set.neuron_ids) / "circuit_config.json"
    c = Circuit(cfg)
    local_nsyn = vpm_nsyn = pom_nsyn = 0
    if "S1nonbarrel_neurons__S1nonbarrel_neurons__chemical" in c.edges.population_names:
        local_nsyn = c.edges["S1nonbarrel_neurons__S1nonbarrel_neurons__chemical"].size
    if "VPM__S1nonbarrel_neurons__chemical" in c.edges.population_names:
        vpm_nsyn = c.edges["VPM__S1nonbarrel_neurons__chemical"].size
    if "POm__S1nonbarrel_neurons__chemical" in c.edges.population_names:
        pom_nsyn = c.edges["POm__S1nonbarrel_neurons__chemical"].size
    print(f"{_inst.neuron_set.neuron_ids}: {c.nodes["S1nonbarrel_neurons"].size} neuron, {local_nsyn} local synapses, {vpm_nsyn} VPM synapses, {pom_nsyn} POm synapses")
